In [3]:
import os
import json
from tqdm import tqdm
import openai
from openai import OpenAI
from typing import List

import re

In [8]:
data = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/aihub_rules.json'
with open(data, 'r', encoding="UTF-8") as j:
    aihub_rule = json.load(j)
    
normal_rule = aihub_rule['normal']
abnormal_rule = aihub_rule['abnormal']

In [7]:
normal_rule = aihub_rule['normal']
abnormal_rule = aihub_rule['abnormal']

NameError: name 'aihub_rule' is not defined

In [11]:
data = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result/H-220607_B16_Y-14_001_0005.json'
with open(data, 'r', encoding="UTF-8") as j:
    caption = json.load(j)

In [14]:
prompt = f"Look at the image caption and decide whether it follows a normal rule or an abnormal rule, and specify what it pertains to.\n\nImage caption: {caption}\n\n normal rule: {normal_rule}\n\n abnormal rule: {abnormal_rule}\n\nRule:(normal/abnormal)\n\nSpecific Rule:"

In [15]:
prompt

'Look at the image caption and decide whether it follows a normal rule or an abnormal rule, and specify what it pertains to.\n\nImage caption: {\'prompt\': \'<image>\\nExplain about the image including Safety net\', \'outputs\': \'<s> The image shows a construction site with a building under construction. The building appears to be at an early stage, with visible scaffolding and what looks like a protective mesh or safety net installed around the site. The safety net is a common safety measure in construction sites to protect workers from falling debris or materials. It is usually installed around the perimeter of the site to prevent accidents and ensure the safety of the workers.\\n\\nThe safety net is made of a strong, durable material that can withstand the weight of construction materials and workers. It is usually installed at a height that is higher than the anticipated fall distance to ensure maximum coverage. The net is usually checked regularly for damage and is often tensione

### BM25

In [ ]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jwsuh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
caption['outputs']

'<s> The image shows a construction site with a building under construction. The building appears to be at an early stage, with visible scaffolding and what looks like a protective mesh or safety net installed around the site. The safety net is a common safety measure in construction sites to protect workers from falling debris or materials. It is usually installed around the perimeter of the site to prevent accidents and ensure the safety of the workers.\n\nThe safety net is made of a strong, durable material that can withstand the weight of construction materials and workers. It is usually installed at a height that is higher than the anticipated fall distance to ensure maximum coverage. The net is usually checked regularly for damage and is often tensioned to maintain its integrity.\n\nThe construction site itself is surrounded by a fence, which is another safety measure to keep unauthorized personnel out of the site and to prevent accidents. The fence is likely to be temporary and 

In [45]:
def tokenize(text):
    return word_tokenize(text.lower())

def find_answer_rank(top_k_indices, answer_index):
    """
    top_k_indices: 상위 문서들의 인덱스가 담긴 배열
    answer_index: 찾고자 하는 정답 문서의 인덱스
    반환값: 정답 문서의 순위 (1부터 시작), 목록에 없는 경우 -1 반환
    """
    # top_k_indices에서 정답 인덱스를 찾아 순위를 반환
    try:
        rank = (top_k_indices == answer_index).nonzero()[0][0] + 1
        return rank
    except IndexError:
        # 정답 인덱스가 배열에 없는 경우
        return -1

In [185]:
root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj'

files = os.listdir(root_path)

# 규칙을 하나의 리스트로 결합
combined_rules = normal_rule + abnormal_rule
# tokenized_rules = [remove_stopwords(sentence) for sentence in combined_rules]

# 규칙 문장들 토큰화
tokenized_rules = [tokenize(rule) for rule in combined_rules]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_rules)

correct = 0

for file in tqdm(sorted(files)):
    # print(file)
    # file = "H-221107_A26_N-05_003_0049.json"
    with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
        caption = json.load(j)
    
    query = tokenize(caption['outputs'])
    # query = remove_stopwords(query)
    # query = ' '.join(query)
    # 상위 k개의 가장 관련있는 규칙 추출
    
    k = 50
    
    top_k = 100  # 예를 들어 상위 10개 결과를 가져오고 싶은 경우
    top_k_scores = bm25.get_scores(query)
    top_k_indices = top_k_scores.argsort()[-top_k:][::-1]  # 점수가 높은 순서로 인덱스 정렬

    # 결과 출력
    top_k_results = [(combined_rules[i], top_k_scores[i]) for i in top_k_indices]
    
    answer = file.split('_')[2]
    
    if answer[0] == 'Y':
        answer_index = int(answer[2:]) - 1
        # print(answer_index)
    elif answer[0] == 'N':
        answer_index = int(answer[2:]) + 49
        # print(answer_index)
    rank = find_answer_rank(top_k_indices, answer_index)
    
    # for i in top_k_results:
    #     print(i)
    
    if rank <= k:
        correct += 1
    # else:
    #     print(f"The answer is ranked {rank} in the top results.")
    # break

print("accuracy:", correct / len(files))
    

 15%|█▍        | 138/943 [00:01<00:08, 92.64it/s]


KeyboardInterrupt: 

In [186]:
top_k_results

[('Working from the top of the ladder', 49.23595092022736),
 ('Worker working at the top and bottom of the ladder', 47.33483903235452),
 ('Working only on the top (or bottom) of the system scaffold',
  44.77276539821859),
 ('Worker operating outside the hazard radius of a dump truck',
  41.54073105577899),
 ('Worker operating within the hazard radius of a dump truck',
  41.54073105577899),
 ('Material placement at the edge of a horse scaffold', 39.91204245925877),
 ('Material placement on the end of the formwork', 39.59277237252375),
 ('Working only on the top (or bottom) during formwork installation and dismantling',
  38.2045953585514),
 ('No materials placed on the end of the formwork', 37.81502574259518),
 ('Worker operating outside the hazard radius on a scissor lift',
  34.67042729158017),
 ('Worker operating within the hazard radius on a scissor lift',
  34.67042729158017),
 ('Worker operating within the hazard radius of a concrete mixer',
  33.2427717079609),
 ('Worker operatin

In [51]:
with open("/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj/H-221027_B13_Y-20_003_0057.json", 'r', encoding="UTF-8") as j:
    caption = json.load(j)

query = tokenize(caption['outputs'])

# 상위 k개의 가장 관련있는 규칙 추출

k = 50

top_k = 100  # 예를 들어 상위 10개 결과를 가져오고 싶은 경우
top_k_scores = bm25.get_scores(query)
top_k_indices = top_k_scores.argsort()[-top_k:][::-1]  # 점수가 높은 순서로 인덱스 정렬

# 결과 출력
top_k_results = [(combined_rules[i], top_k_scores[i]) for i in top_k_indices]

for i in top_k_results:
    print(i)
    

('Worker working at the top and bottom of the ladder', 115.51986914951269)
('Working from the top of the ladder', 114.69374316147665)
('Material placement on the end of the formwork', 105.13267045438994)
('Material placement at the edge of a horse scaffold', 102.60859878476975)
('No materials placed on the end of the formwork', 102.32331257552454)
('Worker operating outside the hazard radius of a dump truck', 94.86321216099593)
('Worker operating outside the hazard radius of a concrete mixer', 94.86321216099593)
('Worker operating within the hazard radius of a concrete mixer', 94.86321216099593)
('Worker operating within the hazard radius of a dump truck', 94.86321216099593)
('Working only on the top (or bottom) of the system scaffold', 93.8928698206872)
('Worker operating outside the hazard radius on a scissor lift', 89.29237759817102)
('Worker operating within the hazard radius on a scissor lift', 89.29237759817102)
('Use of a metal lifting box during mobile crane operations', 87.123

### DPR

In [100]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

In [147]:
# 모델과 토크나이저 로드
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

# 규칙 문장 벡터화 함수
def encode_contexts(texts):
    inputs = context_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    embeddings = context_encoder(**inputs)
    return embeddings.pooler_output

# 쿼리 벡터화 함수
def encode_query(query):
    inputs = question_tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    embeddings = question_encoder(**inputs)
    return embeddings.pooler_output

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

In [106]:
def find_rank_of_answer_in_dpr_results(top_k_indices, answer_index):
    """
    top_k_indices: 상위 문서들의 인덱스가 담긴 배열
    answer_index: 찾고자 하는 정답 문서의 인덱스
    반환값: 정답 문서의 순위 (1부터 시작), 목록에 없는 경우 -1 반환
    """
    try:
        # top_k_indices 배열에서 정답 인덱스의 위치를 찾아 순위 반환
        rank = list(top_k_indices).index(answer_index) + 1
        return rank
    except ValueError:
        # 정답 인덱스가 배열에 없는 경우
        return -1

In [149]:
# 규칙 문장 벡터화
context_embeddings = encode_contexts(combined_rules)

# 쿼리 벡터화
correct = 0

root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj'

files = os.listdir(root_path)

for file in tqdm(sorted(files)):
    # print(file)
    # file = "H-221107_A26_N-05_003_0049.json"
    with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
        caption = json.load(j)
    # query = remove_stopwords(caption['outputs'])
    # query = ' '.join(query)
    query_embedding = encode_query(caption['outputs'])
    # query_embedding = encode_query(query)

    # 유사성 계산
    similarities = torch.matmul(query_embedding, context_embeddings.T).squeeze(0)

    k = 10

    # 상위 k개 문서 반환
    
    top_k = 100
    top_k_indices = torch.topk(similarities, k=top_k).indices

    # 결과 출력
    # for index in top_k_indices:
        # print(combined_rules[index], similarities[index].item())
    answer = file.split('_')[2]
    
    if answer[0] == 'Y':
        answer_index = int(answer[2:]) - 1
        # print(answer_index)
    elif answer[0] == 'N':
        answer_index = int(answer[2:]) + 49
        # print(answer_index)
    rank = find_rank_of_answer_in_dpr_results(top_k_indices, answer_index)
    
    # for i in top_k_results:
    #     print(i)
    
    if rank <= k:
        correct += 1
    # else:
    #     print(f"The answer is ranked {rank} in the top results.")
    # break

print("accuracy:", correct / len(files))

100%|██████████| 943/943 [01:19<00:00, 11.83it/s]

accuracy: 0.3902439024390244


In [118]:
with open("/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj/H-220607_B16_Y-14_001_0005.json", 'r', encoding="UTF-8") as j:
    caption = json.load(j)
query_embedding = encode_query(caption['outputs'])

# 유사성 계산
similarities = torch.matmul(query_embedding, context_embeddings.T).squeeze(0)

k = 50

# 상위 k개 문서 반환

top_k = 100
top_k_indices = torch.topk(similarities, k=top_k).indices

# 결과 출력
for index in top_k_indices:
    print(combined_rules[index], similarities[index].item())


No falling hazards placed on system scaffold 73.15587615966797
Proper horse scaffold installation 71.16910552978516
Falling hazards placed on system scaffold 70.80186462402344
Safety railing installation for openings defect 70.44868469238281
Fire extinguisher placement outside hazardous material storage 70.3277587890625
Material placement at the edge of a horse scaffold 69.73934173583984
Proper placement of materials and tools on a horse scaffold 69.5841293334961
Fire extinguisher placement next to welding equipment 69.04817199707031
Safety device installation during auger crane operations 68.91004943847656
Working only on the top (or bottom) during formwork installation and dismantling 68.57279968261719
No fire extinguisher placement next to welding equipment 68.35093688964844
No fire extinguisher placement with oxy cutter in material storage area 68.17562866210938
Safety net installation 68.1009292602539
rolling tower safety railing installation 68.05232238769531
System scaffold catw

In [159]:
prompt = f"Look at the image caption and decide whether it follows a normal rule or an abnormal rule, and specify what it pertains to.\n\nImage caption: {caption['outputs']}\n\n normal rule: {normal_rule}\n\n abnormal rule: {abnormal_rule}\n\nRule:(normal/abnormal)\n\nSpecific Rule:"

In [161]:
prompt

'Look at the image caption and decide whether it follows a normal rule or an abnormal rule, and specify what it pertains to.\n\nImage caption: <s> The image shows an entrance to a building that appears to be undergoing construction or renovation. The entrance is marked by a temporary guard shelf, which is a safety measure to prevent unauthorized access to the construction site. The guard shelf is designed to be sturdy enough to withstand the weight of construction equipment and workers, while also being low enough to not obstruct the entrance for pedestrians.\n\nThe entrance is marked by a sign with the number "1307" and some Korean text, which could indicate a specific section or level of the building. The text "1307" might refer to a floor number or a code for the construction site. The presence of the guard shelf suggests that the entrance is not yet open for public use, and it\'s likely that the construction work is still ongoing.\n\nThe building itself has a concrete facade, which

In [3]:
data = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/aihub_rules.json'
with open(data, 'r', encoding="UTF-8") as j:
    aihub_rule = json.load(j)
    
normal_rule = aihub_rule['normal']
abnormal_rule = aihub_rule['abnormal']

In [ ]:
save_path = "/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/all_rules"

root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj'

files = os.listdir(root_path)

for file in tqdm(sorted(files)):
    with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
        caption = json.load(j)
    prompt = f"Look at the image caption and decide whether it follows a normal rule or an abnormal rule\n\nImage caption: {caption['outputs']}\n\n Rule: (choose one normal/abnormal)"
    try:
        client = OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
                        model=MODEL_CONFIG['model'],
                        messages=[{"role": "user", "content": prompt}],
                    )
    except Exception as e:
        print(e)

    output = response.choices[0].message.content.lower()
    print(output)
    print("\n==================================\n")
    
    # with open(os.path.join(save_path, file), 'w', encoding="UTF-8") as j:
    #     json.dump(output, j, indent='\t')
    # break
    
    # break

In [ ]:
"Look at the image caption and rules, decide whether situation is normal or abnormal.\n\nImage caption: {caption['outputs']}\n\n Normal Rules: {normal_rule}\n\nAbnormal Rules: {abnormal_rule}Situation: (choose one normal/abnormal)\n\n"

## All rules chatgpt

In [68]:
import os
import json
import multiprocessing
from tqdm import tqdm
from openai import OpenAI

save_path = "/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/no_obj/all_rules"
root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result'
combined_rules = normal_rule + abnormal_rule
def process_file(file):
    try:
        with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
            caption = json.load(j)
        
        prompt = f"Look at the image caption and rules, decide whether situation is normal or abnormal.\n\nImage caption: {caption['outputs']}\n\n Normal Rules: {normal_rule}\n\nAbnormal Rules: {abnormal_rule}\n\nSituation: (choose one normal/abnormal)\n\n"
        client = OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
            model=MODEL_CONFIG['model'],
            messages=[{"role": "user", "content": prompt}],
        )
        output = response.choices[0].message.content.lower()

        with open(os.path.join(save_path, file), 'w', encoding="UTF-8") as j:
            json.dump(output, j, indent='\t')
        
        return f"Processed {file}"

    except Exception as e:
        return str(e)

def main():
    files = os.listdir(root_path)
    pool = multiprocessing.Pool(processes=16)  # Number of worker processes
    for result in tqdm(pool.imap_unordered(process_file, sorted(files)), total=len(files)):
        # print(result)
        # print("\n==================================\n")
        pass

if __name__ == "__main__":
    main()

100%|██████████| 957/957 [00:39<00:00, 24.00it/s]


### No rules chatgpt

In [70]:
import os
import json
import multiprocessing
from tqdm import tqdm
from openai import OpenAI

save_path = "/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/no_obj/no_rules"
root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result'

def process_file(file):
    try:
        # with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
        #     caption = json.load(j)
        
        prompt = f"Look at the image caption, decide whether situation is normal or abnormal.\n\nImage caption: {caption['outputs']}\n\n Situation: (choose one normal/abnormal)\n\n"
        client = OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
            model=MODEL_CONFIG['model'],
            messages=[{"role": "user", "content": prompt}],
        )
        output = response.choices[0].message.content.lower()

        with open(os.path.join(save_path, file), 'w', encoding="UTF-8") as j:
            json.dump(output, j, indent='\t')
        
        return f"Processed {file}"

    except Exception as e:
        return str(e)

def main():
    files = os.listdir(root_path)
    pool = multiprocessing.Pool(processes=16)  # Number of worker processes
    for result in tqdm(pool.imap_unordered(process_file, sorted(files)), total=len(files)):
        # print(result)
        # print("\n==================================\n")
        pass

if __name__ == "__main__":
    main()

100%|██████████| 957/957 [00:36<00:00, 26.35it/s]


### RAG Top-k rules

In [63]:
def classify_rule(rule, normal_rules, abnormal_rules):
    """주어진 규칙을 'normal' 또는 'abnormal'로 분류합니다."""
    if rule in normal_rules:
        return 'normal'
    elif rule in abnormal_rules:
        return 'abnormal'
    else:
        return 'unknown'  # 규칙이 어느 카테고리에도 속하지 않는 경우

In [64]:
normal_rule = aihub_rule['normal']
abnormal_rule = aihub_rule['abnormal']

##### BM25

In [65]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def tokenize(text):
    return word_tokenize(text.lower())

combined_rules = normal_rule + abnormal_rule
tokenized_rules = [tokenize(rule) for rule in combined_rules]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_rules)

[nltk_data] Downloading package punkt to /home/jwsuh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
import os
import json
import multiprocessing
from tqdm import tqdm
from openai import OpenAI

top_k = 50

save_path = f"/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/no_obj/BM25/rag_k_{top_k}"
os.makedirs(save_path, exist_ok=True)
root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result'

def process_file(file):
    try:
        with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
            caption = json.load(j)
        
        query = tokenize(caption['outputs'])
        top_k_scores = bm25.get_scores(query)
        top_k_indices = top_k_scores.argsort()[-top_k:][::-1]  # 점수가 높은 순서로 인덱스 정렬

        # 결과 출력
        top_k_results = [(combined_rules[i], top_k_scores[i]) for i in top_k_indices]
        top_k_results = [item[0] for item in top_k_results]
        
        classified_results = {rule: classify_rule(rule, normal_rule, abnormal_rule) for rule in top_k_results}
        
        normal_group = [rule for rule, category in classified_results.items() if category == 'normal']
        abnormal_group = [rule for rule, category in classified_results.items() if category == 'abnormal']
        
        prompt = f"Look at the image caption and rules, decide whether situation is normal or abnormal.\n\nImage caption: {caption['outputs']}\n\n Normal Rules: {normal_group}\n\nAbnormal Rules: {abnormal_group}\n\nSituation: (choose one normal/abnormal)\n\n"
        client = OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
            model=MODEL_CONFIG['model'],
            messages=[{"role": "user", "content": prompt}],
        )
        output = response.choices[0].message.content.lower()

        with open(os.path.join(save_path, file), 'w', encoding="UTF-8") as j:
            json.dump(output, j, indent='\t')
        
        return output

    except Exception as e:
        return str(e)

def main():
    files = os.listdir(root_path)
    pool = multiprocessing.Pool(processes=16)  # Number of worker processes
    for result in tqdm(pool.imap_unordered(process_file, sorted(files)), total=len(files)):
        # print(result)
        # print("\n==================================\n")
        pass

if __name__ == "__main__":
    main()

100%|██████████| 957/957 [00:41<00:00, 22.80it/s]


##### DPR

In [1]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

/home/jwsuh/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer

# CUDA 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델과 토크나이저 로드
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base').to(device)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

# 규칙 문장 벡터화 함수
def encode_contexts(texts):
    inputs = context_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    embeddings = context_encoder(**inputs)
    return embeddings.pooler_output

# 쿼리 벡터화 함수
def encode_query(query):
    inputs = question_tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
    embeddings = question_encoder(**inputs)
    return embeddings.pooler_output

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

In [ ]:
import os
import json
import multiprocessing
from tqdm import tqdm
from openai import OpenAI

top_k = 10

save_path = f"/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/DPR/rag_k_{top_k}"
os.makedirs(save_path, exist_ok=True)
root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj'

def process_file(file):
    try:
        # print(1)
        with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
            caption = json.load(j)
        # print(1)
        context_embeddings = encode_contexts(combined_rules)
        # print(1)        
        query_embedding = encode_query(caption['outputs'])
        # query_embedding = encode_query(query)
        # print(1)   
        # 유사성 계산
        similarities = torch.matmul(query_embedding, context_embeddings.T).squeeze(0)
        # print(1)   
        top_k_indices = torch.topk(similarities, k=top_k).indices
        # print(1)   
        # 결과 출력
        top_k_results = [combined_rules[index] for index in top_k_indices]
        classified_results = {rule: classify_rule(rule, normal_rule, abnormal_rule) for rule in top_k_results}
        
        normal_group = [rule for rule, category in classified_results.items() if category == 'normal']
        abnormal_group = [rule for rule, category in classified_results.items() if category == 'abnormal']
        # print(1)   
        prompt = f"Look at the image caption and rules, decide whether situation is normal or abnormal.\n\nImage caption: {caption['outputs']}\n\n Normal Rules: {normal_group}\n\nAbnormal Rules: {abnormal_group}\n\nSituation: (choose one normal/abnormal)\n\n"
        client = OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
            model=MODEL_CONFIG['model'],
            messages=[{"role": "user", "content": prompt}],
        )
        output = response.choices[0].message.content.lower()

        with open(os.path.join(save_path, file), 'w', encoding="UTF-8") as j:
            json.dump(output, j, indent='\t')
        
        return output

    except Exception as e:
        return str(e)

def main():
    files = os.listdir(root_path)
    pool = multiprocessing.Pool(processes=16)  # Number of worker processes
    for result in tqdm(pool.imap_unordered(process_file, sorted(files)), total=len(files)):
        print(result)
        # print("\n==================================\n")
        pass

if __name__ == "__main__":
    main()

In [43]:
import os
import json
from openai import OpenAI
from tqdm import tqdm

top_ks = [50, 70]


# save_path = f"/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/DPR/rag_k_{top_k}"
# os.makedirs(save_path, exist_ok=True)
root_path = '/home/jwsuh/harddisk/2403_new_research/construction_dataset/dataset/LLaVA_inference/llava_image_result_with_obj'

def process_file(file, top_k, save_path):
    try:
        with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
            caption = json.load(j)
        
        context_embeddings = encode_contexts(combined_rules)        
        query_embedding = encode_query(caption['outputs'])
        similarities = torch.matmul(query_embedding, context_embeddings.T).squeeze(0)
        top_k_indices = torch.topk(similarities, k=top_k).indices
        top_k_results = [combined_rules[index] for index in top_k_indices]
        classified_results = {rule: classify_rule(rule, normal_rule, abnormal_rule) for rule in top_k_results}
        
        normal_group = [rule for rule, category in classified_results.items() if category == 'normal']
        abnormal_group = [rule for rule, category in classified_results.items() if category == 'abnormal']
        
        prompt = f"Look at the image caption and rules, decide whether situation is normal or abnormal.\n\nImage caption: {caption['outputs']}\n\n Normal Rules: {normal_group}\n\nAbnormal Rules: {abnormal_group}\n\nSituation: (choose one normal/abnormal)\n\n"
        client = OpenAI(api_key=API_KEY)
        response = client.chat.completions.create(
            model=MODEL_CONFIG['model'],
            messages=[{"role": "user", "content": prompt}],
        )
        output = response.choices[0].message.content.lower()

        with open(os.path.join(save_path, file), 'w', encoding="UTF-8") as j:
            json.dump(output, j, indent='\t')
        
        return output

    except Exception as e:
        return str(e)

def main():
    
    files = os.listdir(root_path)
    for top_k in top_ks:
        save_path = f"/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/DPR/rag_k_{top_k}"
        os.makedirs(save_path, exist_ok=True)
        # print(top_k)
        for file in tqdm(sorted(files)):
            # continue
            if file.endswith('.json'):
                result = process_file(file, top_k, save_path)
            # print(result)
            # print("\n==================================\n")

if __name__ == "__main__":
    main()

  2%|▏         | 41/1900 [00:13<10:24,  2.98it/s]


KeyboardInterrupt: 

In [73]:
pattern = r'\bnormal\b'

root_path = "/home/jwsuh/harddisk/2403_new_research/construction_dataset/chatgpt_output/aihub/no_obj/BM25/rag_k_10"
files = os.listdir(root_path)

correct = 0
normals = 0
abnormals = 0

for file in tqdm(files):
    with open(os.path.join(root_path, file), 'r', encoding="UTF-8") as j:
        text = json.load(j)

    if bool(re.search(pattern, text)): # normal
        pred = "normal"
        normals += 1
    else:
        pred = "abnormal"
        abnormals += 1
    
    name = file.split('_')[2]
    
    if name[0] == 'Y':
        answer = 'normal'
        # print(answer_index)
    elif name[0] == 'N':
        answer = "abnormal"

    if answer == pred:
        correct += 1

print("Accuracy:", correct / len(files))
print("Normal:", normals)
print("Abnormal:", abnormals)
# for text, is_normal in results.items():
#     print(f"'{text}' is normal: {is_normal}")

100%|██████████| 957/957 [00:00<00:00, 12572.19it/s]

Accuracy: 0.4994775339602926
Normal: 905
Abnormal: 52
